Before starting....

In [ ]:
import pandas as pd
import numpy as np 

# store CSV into HDF for performance reasons
files = [
    {
        "filename": "bookings.csv", 
        "fields" : ['arr_port', 'dep_port', 'pax'],
        "mapping" : {'arr_port':np.str, 'dep_port':np.str, 'pax':np.float64}
    },
    {
        "filename": "searches.csv", 
        "fields" : ['Origin', 'Destination'], # means all
        "mapping" : {
            'Origin':np.str,
            'Destination':np.str
        }
    }
]

def toHDF(filedata):
    current_key_id = 1
    filename_hdf = "%s.%s" % (filedata["filename"].split(".")[0], "h5")
    print "Processing %s" % filedata["filename"]
    for dataframe_chunk in pd.read_csv(filedata["filename"], sep="^", chunksize=50000, dtype=filedata["mapping"]):
        #trim column names
        dataframe_chunk.columns = map(lambda x : x.strip(), dataframe_chunk.columns)
        if len(filedata["fields"]) > 0:
            dataframe_chunk[filedata["fields"]].to_hdf(filename_hdf, 'data', table=True, append=True)
        else:
            dataframe_chunk.to_hdf(filename_hdf, current_key, table=True, append=True)
            
        current_key_id += 1

    print "Done..."

for f in files:
    toHDF(f)

Exercise 1

In [ ]:
import pandas as pd
import time 

# ex 1
for f in files:
    store = pd.HDFStore(f["filename"].split(".")[0] + '.h5')
    print "%s has %d lines" % (f["filename"].split(".")[0], store.get_storer('data').nrows)

 Exercise 2

In [ ]:
import pandas as pd
import time 
import os.path
from GeoBases import GeoBase

geo_o = GeoBase(data='ori_por', verbose=False)   

def getTopAirports():
    store = pd.HDFStore('bookings.h5')
    df = store.select('data')
    grouped_chunks = df.groupby('arr_port').sum()
    final_dataframe = grouped_chunks.groupby(level=0).sum().sort('pax',ascending=0)
    final_dataframe.to_hdf('top_airports.h5', 'data',format='table')
    return final_dataframe

def getTopNAirports(n):
    if os.path.isfile('top_airports.h5'):
        # already computed. Read from disk
        df = pd.read_hdf('top_airports.h5', 'data', start=0, stop=n)
        df.index = df.index.map(lambda code: geo_o.get(code.strip(), 'name'))
        return df
    else:
        # compute
        df = getTopAirports()[0:n]
        df.index = df.index.map(lambda code: geo_o.get(code.strip(), 'name'))
        return df

getTopNAirports(10)

Exercise 3

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

airport_codes = ['AGP', 'MAD', 'BCN']
airport_names = ['Malaga', 'Madrid', 'Barcellona']
# read the data and extract the month for the selected airports
chunks = []


for c in pd.read_csv('searches.csv', sep="^", chunksize=50000):
    x = c[c["Destination"].isin(airport_codes)]
    x.ix[:, "Month"] = x["Date"].apply(lambda x : x.split("-")[1])
    chunks.append( x[["Destination", "Month"]].groupby(['Destination', 'Month']).size() )
final_dataframe = pd.DataFrame(pd.concat(chunks, axis=1).sum(axis=1))
plt.ylabel('nr. of passengers')
plt.xlabel('months')
plt.xticks(xrange(0,12), xrange(1,12))
plt.title('Monthly searches for Malaga, Madrid, Barcelona')
plt.hold(True)
for i, airport in enumerate(airport_codes):
    plt.plot(final_dataframe.xs(airport))
plt.hold(False)
plt.legend(airport_names)
plt.show()